## Greating EnzymeML document for PPCS:

In [1]:
import glob, pickle, os
import numpy as np

from pyenzyme import (
    EnzymeMLDocument,
    Reactant, Complex, Vessel,
    Protein, EnzymeReaction,
    KineticModel, Measurement,
    Replicate
)

## Initial Set-up of EnzymeML document. Adding of vessels, proteins and reactants:

In [2]:
# Start of the EnzymeMLDocument
enzmldoc = EnzymeMLDocument(name="Phosphopantothenoylcysteine synthetase (PPCS)")

# Adding Vessel:
vessel = Vessel(name="96 well plate", volume=150, unit="ul")
vessel_id = enzmldoc.addVessel(vessel)

# Adding protein and reactants:
Ppan_id = enzmldoc.addReactant(Reactant(
    name = "4'-Phosphopantothenate", vessel_id = vessel_id, init_conc = 1,
    unit = "mM", constant = False, smiles = 'CC(C)(COP(O)(O)=O)[COOH](O)C(=O)NCCC(O)=O',
    chebi_id = 'CHEBI:15905',
))

ctp_id = enzmldoc.addReactant(Reactant(
    name = "Cytidine triphosphate", vessel_id = vessel_id, init_conc = 1,
    unit = "mM", constant = False, smiles = 'NC1CCN((COOH)2O(COH)(COP(O)(=O)OP(O)(=O)OP(O)(O)=O)(COOH)(O)(COH)2O)C(=O)n1',
    chebi_id = 'CHEBI:17677',
))

PPi_id = enzmldoc.addReactant(Reactant(
    name = "Inorganic Pyrophospahte", vessel_id = vessel_id, init_conc = 1,
    unit = "mM", constant = False, smiles = '[O-]P(=O)([O-])OP(=O)([O-])[O-]',
    chebi_id = 'CHEBI:18361',
))

In [3]:
cys_id = enzmldoc.addReactant(Reactant(
    name = "L-cysteine", vessel_id = vessel_id, init_conc = 1,
    unit = "mM", constant = False, smiles = 'N(COOH)(CS)C(O)=O',
    chebi_id = 'CHEBI:17561',
))

CMP_id = enzmldoc.addReactant(Reactant(
    name = "Cytidine monophosphate", vessel_id = vessel_id, init_conc = 1,
    unit = "mM", constant = False, smiles = 'c1cn(c(=O)nc1N)[COH]2[COOH]([COOH]([COH](O2)COP(=O)(O)O)O)O',
    chebi_id = 'CHEBI:20878',
))

Ppan_cys_id = enzmldoc.addReactant(Reactant(
    name = "4'-Phosphopantothenoylcysteine", vessel_id = vessel_id, init_conc = 1,
    unit = "mM", constant = False
))

    
coabc_id = enzmldoc.addProtein(Protein(
    name="CoaBC", sequence = "UNKNOWN", vessel_id=vessel_id, 
    organism="Staphylococcus aureus", init_conc=0.05, unit="mg / ml",
))

## Adding the reaction to the EnzymeML document: 

In [4]:
# Creating the reaction:

reaction1 = EnzymeReaction(temperature=25.0, temperature_unit="C", 
                           ph=7.8, name="Phosphopantothenoylcysteine synthetase", reversible=False)

reaction1.addEduct(Ppan_id, 1.0, enzmldoc)
reaction1.addEduct(ctp_id, 1.0, enzmldoc)
reaction1.addProduct(PPi_id, 1.0, enzmldoc)
reaction1.addModifier(coabc_id, 1.0, enzmldoc, False)
reaction1.addEduct(cys_id , 1.0, enzmldoc)
reaction1.addProduct(CMP_id , 1.0, enzmldoc)
reaction1.addProduct(Ppan_cys_id , 1.0, enzmldoc)



In [5]:
# Add the reactions to the  EnzymeML document

enzmldoc.addReactions([reaction1])

{'Phosphopantothenoylcysteine synthetase': 'r0'}

## Adding  measurements 

Create measurment for first condition at CTP = 1mM, PPan = 0.37 mM and Cys = 1mM

In [6]:
import numpy as np
import pandas as pd

# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_1mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_1mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-CTP_1mM.xlsx")

# Create the Measurement object for one run at CTP = 1 mM
measurement_1 = Measurement(
    name="Run 1 - CTP = 1mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_1.addData(reactant_id=Ppan_id, init_conc=0.35, unit="mM")
measurement_1.addData(reactant_id=ctp_id, init_conc=1, unit="mM")
measurement_1.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_1.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

In [7]:
# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 1 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['B2'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 1 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['B4'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 1 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['B2'].tolist()
)

# Add replicates to measurement
measurement_1.addReplicates(repeat1, enzmldoc)
measurement_1.addReplicates(repeat2, enzmldoc)
measurement_1.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_1_id = enzmldoc.addMeasurement(measurement_1)

print("Measurement created for three repeats at CTP = 1 mM")

# Check the measurement table
print(enzmldoc.printMeasurements())

Measurement created for three repeats at CTP = 1 mM
>>> Measurement m0: Run 1 - CTP = 1mM
    s0 | initial conc: 0.35 mmole / l 	| #replicates: 0
    s1 | initial conc: 1.0 mmole / l 	| #replicates: 3
    s3 | initial conc: 1.0 mmole / l 	| #replicates: 0
    p0 | initial conc: 0.05 mg / ml 	| #replicates: 0
None


Create measurment for second condition at CTP = 1 mM, Ppan = 0.18 mM and Cys = 1 mM

In [8]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_1mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_1mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-CTP_1mM.xlsx")

# Create the Measurement object for one run at CTP = 1 mM
measurement_2 = Measurement(
    name="Run 2 - CTP = 1mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_2.addData(reactant_id=Ppan_id, init_conc=0.18, unit="mM")
measurement_2.addData(reactant_id=ctp_id, init_conc=1, unit="mM")
measurement_2.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_2.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 1 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['C2'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 1 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['C4'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 1 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['C2'].tolist()
)

# Add replicates to measurement
measurement_2.addReplicates(repeat1, enzmldoc)
measurement_2.addReplicates(repeat2, enzmldoc)
measurement_2.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_2_id = enzmldoc.addMeasurement(measurement_2)

print("Measurement created for three repeats at CTP = 1 mM, Ppan = 0.18 mM and Cys = 1mM")

Measurement created for three repeats at CTP = 1 mM, Ppan = 0.18 mM and Cys = 1mM


In [9]:
# Check the measurement table
print(enzmldoc.printMeasurements())

>>> Measurement m0: Run 1 - CTP = 1mM
    s0 | initial conc: 0.35 mmole / l 	| #replicates: 0
    s1 | initial conc: 1.0 mmole / l 	| #replicates: 3
    s3 | initial conc: 1.0 mmole / l 	| #replicates: 0
    p0 | initial conc: 0.05 mg / ml 	| #replicates: 0
>>> Measurement m1: Run 2 - CTP = 1mM
    s0 | initial conc: 0.18 mmole / l 	| #replicates: 0
    s1 | initial conc: 1.0 mmole / l 	| #replicates: 3
    s3 | initial conc: 1.0 mmole / l 	| #replicates: 0
    p0 | initial conc: 0.05 mg / ml 	| #replicates: 0
None


## Add Kinetic Model


In [10]:
# Create models py parsing equations

kinmod_PPCS = KineticModel.fromEquation("IMM_PPCS", 
                             "(vf*s1*s0*s3)/((K_ictp*K_ppan*s3)+(K_cys*s1*s0)+(K_ppan*s1*s3)+(K_ctp*s0*s3)+(s1*s0*s3))")

In [11]:
# Configure parameters for models

kinmod_PPCS._vf.update(value=13.32, unit="mM / min", lower=1e-6)
kinmod_PPCS._K_ictp.update(value=0.019, unit="mM", lower=1e-6)
kinmod_PPCS._K_ctp.update(value=0.64, unit="mM", lower=1e-6)
kinmod_PPCS._K_ppan.update(value=0.161, unit="mM", lower=1e-6)
kinmod_PPCS._K_cys.update(value=0.161, unit="mM", lower=1e-6)


In [12]:
# Add models to each reaction

reaction1.model = kinmod_PPCS

In [13]:
# Finally, write to EnzymeML
enzmldoc.toFile(".", "Enzyme ML doc")


Archive was written to .\Enzyme_ML_doc.omex



In [14]:
import numpy as np
from pyenzyme import EnzymeMLDocument

In [15]:
enzmldoc

Phosphopantothenoylcysteine synthetase (PPCS)
>>> Reactants
	ID: s0 	 Name: 4'-Phosphopantothenate
	ID: s1 	 Name: Cytidine triphosphate
	ID: s2 	 Name: Inorganic Pyrophospahte
	ID: s3 	 Name: L-cysteine
	ID: s4 	 Name: Cytidine monophosphate
	ID: s5 	 Name: 4'-Phosphopantothenoylcysteine
>>> Proteins
	ID: p0 	 Name: CoaBC
>>> Complexes
>>> Reactions
	ID: r0 	 Name: Phosphopantothenoylcysteine synthetase